<a href="https://colab.research.google.com/github/compi1234/pyspch/blob/master/test/spectrogram_test_ly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Spectrogram Plotting with plotly

Requirements:   
- plotly   
- pyspch   

In [ ]:
# You may need to install following packages
# !pip install plotly=4.12
# !pip install git+https://github.com/compi1234/pyspch.git
# !pip install pydub
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

In [ ]:
import os,sys 
import numpy as np
import pandas as pd
from IPython.display import display, Audio, HTML, clear_output
import ipywidgets as widgets

import plotly 
import plotly.graph_objects as go
from plotly.subplots import make_subplots
#
import pyspch.spectrogram as specg
import pyspch.audio as audio
import pyspch.utils as spch
import pyspch.display_ly as spch_disp

In [ ]:
print(plotly.__version__)
# you may need to set a new default renderer
# in Jupyter lab the automatic detection doesn't work flawless
# it starts up with 'plotly_mimetype+notebook'
import plotly.io as pio
#pio.renderers.default='notebook'
#pio.renderers.default='jupyterlab'
#pio.renderers.default='plotly_mimetype'
pio.renderers.default

In [ ]:
# set basename and directories
dir='https://homes.esat.kuleuven.be/~spchlab/data/'
file = "timit/si1027" #@param {type:"string"}
basename = dir+file
wavfname = basename + ".wav"
wrdfname = basename + ".wrd"
phnfname = basename + ".phn"
phone_mapping = "timit/phones-61-48-39.txt" #@param {type:"string"}

# 1. read the waveform data
wavdata, sr = audio.load(wavfname)

# 2. compute spectrogram
f_shift = 0.01
spgdata = specg.spectrogram(wavdata,sample_rate=sr,f_shift=f_shift,n_mels=None)
(nparam,nfr) = spgdata.shape
spg_xaxis = (np.arange(nfr)+0.5)*f_shift
spg_yaxis = np.arange(nparam)*sr/(2.*(nparam-1))

# 3. get segmentations
segwrd = pd.read_csv(wrdfname,delim_whitespace=True,names=['t0','t1','seg'])
segphn61 = pd.read_csv(phnfname,delim_whitespace=True,names=['t0','t1','seg'])
# convert TIMIT61 to TIMIT39
# this routine is not great as it simply maps closures to silence instead to combining
phone_tbl = pd.read_csv(dir+phone_mapping,delim_whitespace=True,names=['T61','T48','T39'])
map61_48 = dict(zip(phone_tbl['T61'],phone_tbl['T48']))
segphn = segphn61.replace({"seg": map61_48})
# convert TIMIT sample based segmentations to time based segmentations
segphn['t0']=specg.indx2t(segphn['t0'],1./sr)
segphn['t1']=specg.indx2t(segphn['t1'],1./sr)
segwrd['t0']=specg.indx2t(segwrd['t0'],1./sr)
segwrd['t1']=specg.indx2t(segwrd['t1'],1./sr)

### 1. Elementary Waveform and Heatmap plotting using plotly

In [ ]:
wave_go = go.Scatter(y=wavdata, dx=1./sr,
                  showlegend=False,
                  hoverinfo="x+y",
                 )
go.Figure(wave_go).show()

In [ ]:
%%time
spg_go = go.Heatmap(z=spgdata,dx=f_shift,x0=f_shift/2.,dy=sr/(2*(nparam-1)),
                 colorscale='Jet',
                 showscale=False,
                 showlegend=False,
                 hoverinfo="x+y+z",
                 name='Spectrogram',
                 text=spgdata)
go.Figure(spg_go).show()

### 2. A high level API for plotting waveforms, segmentations and spectrograms 
using pyspch.display_ly.plot_waveform() and pyspch.display_ly.plot_spg()   
in their most basic usage, these functions are identical as in pyspch.display (using matplotlib)

In [ ]:
%%time
fig = spch_disp.plot_waveform(wavdata, sample_rate=sr)
fig.show()

In [ ]:
fig = spch_disp.plot_waveform(wavdata, sample_rate=sr,seg=segwrd,ypos=.85,title="Waveform + Transcription")
fig.show()

In [ ]:
spch_disp.add_seg_plot(fig,segphn,ypos=.1,textfont={'color':'black','size':16})
fig.update_layout(title="Waveform + Transcript + Phone Labels")
#fig = spch_disp.plot_waveform(wavdata, sample_rate=sr,title="Waveform + Phone labels",seg=segphn,ypos=.1)
#add word transcription as well
#spch_disp.add_seg_plot(fig,segwrd,ypos=.85)
display(fig)

In [ ]:
%%time
fig = spch_disp.plot_spg(spgdata,wav=wavdata,sample_rate=sr,f_shift=f_shift)
fig.show()

In [ ]:
%%time
fig = spch_disp.plot_spg(spgdata,wav=wavdata,sample_rate=sr,f_shift=f_shift,segwav=segwrd,segspg=segphn)
fig.show()

### 3. Interactive Spectrogram with Frame Sliders

In [ ]:

def box_layout():
     return widgets.Layout(
        border='solid 1px black',
        margin='0px 10px 10px 0px',
        padding='5px 5px 5px 5px'
     )

class int_specg(widgets.VBox):
    def __init__(self,spg,wavdata,f_shift=0.01,sample_rate=16000,dpi=100):
        super().__init__()
        (nparam,nfr)=spg.shape
        self.sample_rate = sample_rate
        self.f_shift = f_shift
        self.n_shift = int(f_shift*self.sample_rate)
        self.nfr = nfr
        self.frames = [0, self.nfr]
        self.times = [0., self.nfr*self.f_shift]
        self.spg = spg
        self.wavdata = wavdata
        self.autoplay = False
        self.dpi = dpi

        self.wg_range = widgets.FloatRangeSlider(value=self.times,min=self.times[0],max=self.times[1],step=self.f_shift,
                            description='Time Selection: ',continuous_update=False)
        self.wg_range.layout.width = str(6*dpi)+'px'
        self.wg_range.observe(self.range_observe,'value')
        self.wg_autoplay = widgets.Checkbox(value=self.autoplay,description='Play Selection',indent=False,button_style='warning')
        self.wg_autoplay.layout.width=str(2*dpi)+'px'
        self.wg_autoplay.observe(self.autoplay_observe, 'value')
        
        self.out = widgets.Output(layout=box_layout())
        self.audio = widgets.Output()
        self.UI = widgets.VBox( [self.wg_range, 
                                 widgets.HBox([self.wg_autoplay,self.audio ]) ],
                                 layout=box_layout())
        self.children = [ self.out , self.UI ]
        
        self.fig = spch_disp.plot_spg(self.spg,wav=self.wavdata,sample_rate=self.sample_rate,frames=self.frames,dpi=self.dpi)
        self.update(INIT=True)
        #self.figinit = self.fig
        
    def update(self,INIT=False):
        if not INIT:
            spch_disp.plot_spg(self.spg,fig=self.fig,wav=self.wavdata,sample_rate=self.sample_rate,
                       frames=self.frames)
        with self.out:
            #print("Should be displaying")
            clear_output(wait=True)
            #self.fig.show()
            display(self.fig)
        with self.audio:
            clear_output(wait=True)
            sample_range = [int(self.times[0]*self.sample_rate),int(self.times[1]*self.sample_rate)]
            display(Audio(data=self.wavdata[sample_range[0]:sample_range[1]],rate=self.sample_rate,autoplay=self.autoplay))
            
    def range_observe(self,change):
        self.times = change.new
        self.frames = [int(self.times[0]/self.f_shift), int(self.times[1]/self.f_shift)]
        self.update()
    def autoplay_observe(self,obj):
        self.autoplay = not self.autoplay
    

In [ ]:
int_specg(spgdata,wavdata,sample_rate=sr,dpi=72)

# JUNK - DEV - WORK

Comments on the plotly framework

1. Generate the subplot framework
    - make a cellgrid with a specified number of (rows,cols)   
    - You can span single subplots over a range of cells   
2. Add traces   
    - refer to cells in the grid (counting starts at 1 !!!!)
    - when using a plot over multiple cells, give position (TOP,LEFT) 
    - For adding layout features: refer to the row + col number of the plot ..   

Known Limitations: 
- no event listener to get the rangeslider boundaries. e.g. for selective audio play
- subplots: some layout features, such as background color, can not be controlled at the subplot level

### Rangeslider
Adding a Rangeslider is interesting in many situations.  However, it is not without shortcomings and errors in operation

In [ ]:
%%time
Rangeslider=True
fig = spch_disp.plot_spg(spgdata,wav=wavdata,sample_rate=sr,f_shift=f_shift,segwav=segwrd,segspg=segphn)
if(Rangeslider):
    fig['layout']['xaxis'].update(title='Time(sec)',
                               type='linear',
                               rangeslider=dict(visible=True,borderwidth=2,bgcolor="#bbb"), 
                               showgrid=False)
fig.show()

In [ ]:
fig = make_subplots(rows=3, cols=1, row_heights=[.3,.6,.1],
                   shared_xaxes=True,
                   vertical_spacing=0.02,
                   start_cell='bottom-left')
fig.update_xaxes(showgrid=False,range=[wavtime[0],wavtime[nsamples-1]])
fig.update_yaxes(showgrid=False,fixedrange=True,zeroline=False)


fig.add_trace(wave_go,1,1)
fig.add_trace(segwrd_go1,1,1)
fig.update_yaxes(row=1,title_text='Amplitude',range=[-wavrange,wavrange],zeroline=True)
for iseg in range(0,len(segwrd)):
    fig.add_vline(row=1,x=segwrd['t0'][iseg], line_dash='dash',line_color='blue')
    fig.add_vline(row=1,x=segwrd['t1'][iseg], line_dash='dot',line_color='blue')
    
fig.add_trace(spg_go,2,1)
fig.update_yaxes(row=2,title_text='Frequency(hz)')

fig.add_trace(segphn_go1,3,1)
fig.update_yaxes(row=3,showticklabels=False)
for iseg in range(0,len(segphn)):
    fig.add_vline(row=3,x=segphn['t0'][iseg], line_dash='dash',line_color='black')
    fig.add_vline(row=3,x=segphn['t1'][iseg], line_dash='dot',line_color='black')
    
fig.update_layout(height=800,width=1000,title_text="Waveform + Spectrogram + Segmentation",selectdirection="h",plot_bgcolor="#f0ffff")
#fig['layout'].update(selectdirection="h",plot_bgcolor="#f0ffff")
fig['layout']['yaxis'].update(fixedrange=True,type='linear')
fig['layout']['xaxis2'].update(title='Time(sec)',
                               type='linear',
                               rangeslider=dict(visible=True,borderwidth=2,bgcolor="#bbb",range=[wavtime[0],wavtime[nsamples-1]]), 
                               showgrid=False)
fig.show()
display(Audio(data=wavdata,rate=sr))



In [ ]:
fig['layout']['yaxis2'].range

In [ ]:
fig.update_xaxes(showgrid=False,range=[wavtime[0],wavtime[nsamples-1]])
fig.update_yaxes(showgrid=False,fixedrange=True,zeroline=False)


fig.add_trace(wave_go,1,1)
fig.add_trace(segwrd_go1,1,1)
fig.update_yaxes(row=1,title_text='Amplitude',range=[-wavrange,wavrange],zeroline=True)
for iseg in range(0,len(segwrd)):
    fig.add_vline(row=1,x=segwrd['t0'][iseg], line_dash='dash',line_color='blue')
    fig.add_vline(row=1,x=segwrd['t1'][iseg], line_dash='dot',line_color='blue')
    
fig.add_trace(spg_go,2,1)
fig.update_yaxes(row=2,title_text='Frequency(hz)')

fig.add_trace(segphn_go1,3,1)
fig.update_yaxes(row=3,showticklabels=False)
for iseg in range(0,len(segphn)):
    fig.add_vline(row=3,x=segphn['t0'][iseg], line_dash='dash',line_color='black')
    fig.add_vline(row=3,x=segphn['t1'][iseg], line_dash='dot',line_color='black')
    
fig.update_layout(height=800,width=1000,title_text="Waveform + Spectrogram + Segmentation",selectdirection="h",plot_bgcolor="#f0ffff")
#fig['layout'].update(selectdirection="h",plot_bgcolor="#f0ffff")
fig['layout']['yaxis'].update(fixedrange=True,type='linear')
fig['layout']['xaxis'].update(title='Time(sec)',
                               type='linear',
                               rangeslider=dict(visible=True,borderwidth=2,bgcolor="#bbb",range=[wavtime[0],wavtime[nsamples-1]]), 
                               showgrid=False)
fig.show()
display(Audio(data=wavdata,rate=sr))

In [ ]:
fig.layout.width=500

In [ ]:
pio.renderers

In [ ]:
import plotly.io as pio
pio.renderers.default='jupyterlab'

In [ ]:
xxx=[1.,2,.3]
xxx[0],xxx[-1]